# Feature engineering, xgboost
https://www.kaggle.com/dlarionov/feature-engineering-xgboost


-　DateNum作成 'date_block_num',→YYYYMMで代用
- 祝日フラグ
- month
- days
- アイテムカテゴリ（デフォルト)-> カテゴリCategory、ジャンルType
- 月の売り上げ
- 月の売り上げラグ 1,2,3,6,12
- アイテムあたりの「週Or日」平均販売個数ラグ1，2，3，6，12　　'date_item_avg_item_cnt_lag_1',
- ショップあたり「週Or日」平均販売個数ラグ1，2，3，6，12　　'date_shop_avg_item_cnt_lag_1',
- アイテムカテゴリあたり「週Or日」平均販売個数ラグ1，2，3，'date_cat_avg_item_cnt_lag_1',
- カテゴリあたり「週Or日」平均販売個数ラグ1，2，3，'date_cat_avg_item_cnt_lag_1',
- カテゴリ/ショップあたり[月] 平均販売個数ラグ1，2，3，
- ジャンル　あたり[月] 平均販売個数ラグ1，2，3，
- ジャンル/ショップ　あたり[月] 平均販売個数ラグ1，2，3，
- ASP　月当たり
- 価格下落率
- 最終販売日
- 初販売日
- ショップあたりの最終販売日
- ショップあたりの初販売日
    'delta_price_lag',


In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:


#データの読込
sales = pd.read_csv('./sales_history.csv')
test = pd.read_csv('./test.csv')

#sales_historyのデータを月次にリサンプリング
sales['日付'] = pd.to_datetime(sales['日付'])
sales['年'] = sales['日付'].dt.year
sales['月'] = sales['日付'].dt.month

sales_month = sales.groupby(['日付','年', '月', '店舗ID', '商品ID'], as_index=False).sum()

#月次売上データから重点予測対象商品170点の商品のみを抽出
important_items = test[['商品ID', '店舗ID']]
predict_items_df = pd.merge(sales_month, important_items, on=['商品ID', '店舗ID'])

#2018-01 ~ 2019-10までの結合用のDataFrameを作成
month_2018 = np.arange(1, 13)
month_2019 = np.arange(1, 11)

df_2018 = pd.DataFrame(month_2018, columns=['月'])
df_2018['年'] = 2018

df_2019 = pd.DataFrame(month_2019, columns=['月'])
df_2019['年'] = 2019

#欠損値補完の準備

"""
base_df: 結合の土台にするDataFrame 【dataframe】
month_sales_df: 補完後のDataFrameを追加していくためのDataFrame 【dataframe】
shop_id: 0~17の店舗ID 【array】
items: 重点予測対象の商品170点の商品ID 【array】
tmp_df: 計算のために使う一時的なDataFrame 【dataframe】
"""

base_df = pd.concat([df_2018, df_2019], axis=0)[['年', '月']]
month_sales_df = pd.DataFrame()

shop_id = np.arange(0, 18)
items = important_items['商品ID'].unique()

#欠損値補完処理(処理時間計測開始)
start_time = time.time()

for id in shop_id:
    for item in items:
        tmp_df = base_df.copy()
        tmp_df['店舗ID'] = id
        tmp_df['商品ID'] = item
        item_df = predict_items_df[(predict_items_df['商品ID'] == item) & 
                                   (predict_items_df['店舗ID'] == id)].drop(columns='店舗ID')
        tmp_df = pd.merge(tmp_df, item_df, on=['年', '月', '商品ID'], how='left').fillna(0)
        month_sales_df = month_sales_df.append(tmp_df)

end_time = time.time()
#計測終了

elasped_time = end_time - start_time

print('処理にかかった時間: {:.3f}秒'.format(elasped_time))

処理にかかった時間: 26.435秒


In [8]:
month_sales_df1 = month_sales_df.drop(['年','月'],axis=1)
month_sales_df1

,店舗ID,商品ID,日付,商品価格,売上個数
0,0,1000001,2018-01-06 00:00:00,420.0,1.0
1,0,1000001,2018-01-07 00:00:00,840.0,3.0
2,0,1000001,2018-01-20 00:00:00,420.0,1.0
3,0,1000001,2018-01-28 00:00:00,420.0,1.0
4,0,1000001,2018-02-07 00:00:00,420.0,1.0
...,...,...,...,...,...
17,17,3500001,0,0.0,0.0
18,17,3500001,0,0.0,0.0
19,17,3500001,0,0.0,0.0
20,17,3500001,0,0.0,0.0


In [10]:
category_names = pd.read_csv('category_names.csv')
category_names.columns = ['ItemCategoryId', 'ItemCategoryName']

item_categories = pd.read_csv('item_categories.csv')
item_categories.columns = ['ItemId', 'ItemCategoryId']

#sales_history = pd.read_csv('sales_history.csv')
sales_history = month_sales_df1
sales_history.columns = ['StoreId', 'ItemId','Date',  'ItemPrice', 'SoldNum']
#sales_history['Date'] = pd.to_datetime(sales_history['Date'])

test = pd.read_csv('test.csv')
test.columns = ['index', 'ItemId', 'StoreId']
#item_categories['ItemCategoryId'].unique()
#category_names.query('ItemCategoryName.str.contains("")')
'''170品目のリスト'''
test_item = test['ItemId'].unique()
test_item = pd.DataFrame({'ItemId':test_item})
#test_item
'''カテゴリ名と連結'''
ttest = pd.merge(test,item_categories, on='ItemId')
ttest = pd.merge(ttest, category_names,on='ItemCategoryId')
print('予測データの品目数: ',ttest['ItemId'].nunique(),'\n',
      '訓練データの品目数: ',sales_history['ItemId'].nunique(),'\n',
      '予測データのカテゴリ数: ',item_categories['ItemCategoryId'].nunique(),'\n',
      '訓練データのカテゴリ数: ',ttest['ItemCategoryId'].nunique(),'\n',
     '売上の総数: ',sales_history['SoldNum'].sum(), '\n',
     '')

予測データの品目数:  170 
 訓練データの品目数:  170 
 予測データのカテゴリ数:  26 
 訓練データのカテゴリ数:  26 
 売上の総数:  50595.0 
 


# カテゴリ・ジャンル作成
# sklearn Labelencoding

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
#ItemCategoryNameを　「ー」で分割
cat_type = category_names['ItemCategoryName'].str.split('-',expand=True)
#カラム名変更
cat_type['Category'] = cat_type[0]
cat_type['Type'] = cat_type[1]
# ItemCategoryId追加
cat_type =pd.concat([cat_type,category_names['ItemCategoryId']],axis=1)
cat_type.rename(columns={0:'CatName',1:'TypeName'},inplace=True)
'''PCゲームをゲームに'''
cat_type.at[1,'Category'] = 'ゲーム'
cat_type.at[1,'CatName'] = 'ゲーム'
''' Label Encoding'''
cat_type.Category = le.fit_transform(cat_type['Category'].values)
cat_type.Type = le.fit_transform(cat_type['Type'].values)

'''testデータにカテゴリ特徴追加'''
pd.merge(ttest,cat_type,on='ItemCategoryId').drop(['CatName','TypeName'],axis=1)

'''全売り上げデータにカテゴリ特徴追加'''
sales_total = pd.merge(sales_history,item_categories,on='ItemId')
sales_total = pd.merge(sales_total,cat_type,on='ItemCategoryId')
#sales_total.groupby('ItemCategoryName').count()
sales_total.head()

,StoreId,ItemId,Date,ItemPrice,SoldNum,ItemCategoryId,CatName,TypeName,Category,Type
0,0,1000001,2018-01-06 00:00:00,420.0,1.0,100,映画,DVD,5,1
1,0,1000001,2018-01-07 00:00:00,840.0,3.0,100,映画,DVD,5,1
2,0,1000001,2018-01-20 00:00:00,420.0,1.0,100,映画,DVD,5,1
3,0,1000001,2018-01-28 00:00:00,420.0,1.0,100,映画,DVD,5,1
4,0,1000001,2018-02-07 00:00:00,420.0,1.0,100,映画,DVD,5,1


# 休日フラグ  sales_total.csvとして保存
https://upura.hatenablog.com/entry/2018/12/21/070000

In [12]:
#sales_total = pd.read_csv('sales_total.csv')

from datetime import date,timedelta
import jpholiday
%time
sales_total['is_holiday'] = sales_total['Date'].map(jpholiday.is_holiday).astype(int)


Wall time: 0 ns


AttributeError: 'int' object has no attribute 'month'

In [5]:
sales_total

,Date,StoreId,ItemId,ItemPrice,SoldNum,ItemCategoryId,CatName,TypeName,Category,Type,is_holiday
0,2018-01-01,9,1000001,420,1.0,100,映画,DVD,5,1,1
1,2018-01-01,6,1000001,420,1.0,100,映画,DVD,5,1,1
2,2018-01-01,10,1000001,420,1.0,100,映画,DVD,5,1,1
3,2018-01-02,10,1000001,420,2.0,100,映画,DVD,5,1,0
4,2018-01-02,9,1000001,420,1.0,100,映画,DVD,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1119565,2019-10-20,6,3300319,3400,1.0,330,音楽,レコード,7,17,0
1119566,2019-10-20,0,3300319,3400,1.0,330,音楽,レコード,7,17,0
1119567,2019-10-25,0,3300323,9350,1.0,330,音楽,レコード,7,17,0
1119568,2019-10-28,1,3300325,2120,1.0,330,音楽,レコード,7,17,0


In [7]:
sales_total.to_csv('sales_total.csv',index=False)